In [1]:
import pandas as pd


import plotly.plotly as py
from plotly.graph_objs import *

import networkx as nx

import network_vis

from ayasdi.core.api import Api
connection = Api(username='myazdani@ucsd.edu',password='lKFvW93gqcE=!')

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


In [2]:
source = connection.get_source(name='table-kegg-clean-transpose.csv 1')

In [3]:
[i['name'] for i in source.get_comparisons()]

[u'subject.type = HE vs. Rest']

## Simple cohort comparison between healthy and not healthy

In [4]:
comparison = source.get_comparison('subject.type = HE vs. Rest')

df = pd.DataFrame(comparison['continuous_explainers'])

df = (df[["name", 'ks_score', 'ks_sign', 'p_value', 't_test_p_value', 'diff_means']]
 .sort_values("ks_score", axis = 0, ascending = False))

df.head()

,name,ks_score,ks_sign,p_value,t_test_p_value,diff_means
1889,K00867.type_I_pantothenate_kinase.,0.971429,-,3.613776e-13,9.208640e-04,-0.000070
3535,K04108.4.hydroxybenzoyl.CoA_reductase_subunit_...,0.942857,-,1.917799e-12,1.018815e-05,-0.000078
6713,K00721.dolichol.phosphate_mannosyltransferase.,0.942857,+,1.917799e-12,4.795360e-14,0.000444
2300,K03753.molybdopterin.guanine_dinucleotide_bios...,0.935714,-,2.575939e-12,1.185121e-06,-0.000066
2966,K13641.IclR_family_transcriptional_regulator._...,0.935714,-,2.575939e-12,1.708861e-04,-0.000159


In [9]:
df.to_csv("../results/kegg-pca-list.csv", index = False)

In [14]:
network = source.get_network("automatic")


In [5]:

node_positions = {}
for node in network.get_nodes():
    node_positions[int(node['id'])] = [node['x'], node['y']]

edges = []
for edge in network.get_edges():
    edges.append((edge["from"], edge["to"]))
G=nx.from_edgelist(edges)

pos = node_positions
G.add_nodes_from(pos.keys())
for n, p in pos.iteritems():
    G.node[n]['pos'] = p

In [22]:
fig = network_vis.graph(G, node_positions, 
                        network.get_coloring_values('subject.type = HE'), 
                        {0: "Not Healthy", 1: "Healthy"}, 
                        chart_title = "Healthy vs Not Healthy groups", 
                        annotated_text = "MDS Lens and Norm Correlation Metric")

py.iplot(fig)

In [7]:
network.get_coloring_values('HE subgroup 1 MDS norm correlation')

fig = network_vis.graph(G, node_positions, 
                        network.get_coloring_values('HE subgroup 1 MDS norm correlation'), 
                        {0: "Not in Healthy Group 1", 1: "Healthy Group 1"})

py.iplot(fig)

In [8]:
fig = network_vis.graph(G, node_positions, 
                        network.get_coloring_values('HE subgroup 2 MDS norm correlation'), 
                        {0: "Not in Healthy Group 2", 1: "Healthy Group 2"})

py.iplot(fig)